## A simple model for demand and supply of publicly-provided services in a city

In [ ]:
from enum import Enum
import os.path

import numpy as np
import pandas as pd
import geopandas as gpd
import geopy, geopy.distance
import shapely
from sklearn import gaussian_process

from matplotlib import pyplot as plt 
import seaborn as sns
plt.rcParams['figure.figsize']= (20,14)

In [ ]:
## TODO: find way to put this into some global settings
import os
import sys
nb_dir = os.path.dirname(os.getcwd())
if nb_dir not in sys.path:
    sys.path.append(nb_dir)

from references import common_cfg

In [ ]:
from src.models.city_items import AgeGroup, ServiceArea, ServiceType, SummaryNorm # enum classes for the model

In [ ]:
from src.models.services_supply import ServiceUnit, ServiceEvaluator, UnitFactory, SchoolFactory, LibraryFactory 
from src.models.services_supply import get_random_pos #TODO refactor this fun into common utils

from src.models.demand import DemandUnit, DemandUnitFactory

from src.models.process_tools import MappedPositionsFrame, ServiceValues, GridMaker, ValuesPlotter

In [ ]:
quicktest = [ServiceUnit(ServiceType.PoliceStation, 'Duomo', ageDiffusionIn=None), 
        ServiceUnit(ServiceType.PoliceStation, 'Ripamonti', 
                    position=geopy.Point(45.43, 9.201), ageDiffusionIn=None)]
ServiceEvaluator(quicktest).evaluate_services_at(MappedPositionsFrame(get_random_pos(4)))
del quicktest

In [ ]:
## Load scuole
scuoleFile =  '../data/processed/Milano_datiScuole.csv'
schoolLoader = UnitFactory.createLoader(ServiceType.School, scuoleFile)

# Initialise with a default lengthscale of 0.5 km
schoolUnits = schoolLoader.load(meanRadius=0.5)
schoolEval = ServiceEvaluator(schoolUnits)

In [ ]:
## Load scuole
bibliotecheFile =  '../data/processed/Milano_biblioteche.csv'
bibliotecheLoader = UnitFactory.createLoader(ServiceType.Library, bibliotecheFile)

# Initialise with a default lengthscale of 0.5 km
libraryUnits = bibliotecheLoader.load(meanRadius=0.5)

In [ ]:
allEvaluator = ServiceEvaluator(schoolUnits + libraryUnits)
testEvaluator = ServiceEvaluator(schoolUnits[0:10])

In [ ]:
def process_city_demand(cityName):
    assert cityName in common_cfg.cityList, 'Unrecognized city name %s' % cityName
    cityDemandFactory = DemandUnitFactory(cityName)
    unitsOut = cityDemandFactory.make_units_at_centroids()
    return unitsOut

unitsOut = process_city_demand('Milano')

In [ ]:
# call grid making to discretise service evaluation, this is an alternative to evaluating on the demand units
milanoGridMK = GridMaker({'quartieri':'../data/raw/Milano_specific/Milano_quartieri.geojson'}, gridStep=.5)

In [ ]:
# compute service levels
#valuesGrid = testEvaluator.evaluate_services_at(milanoGridMK.grid)
valuesGrid = testEvaluator.evaluate_services_at(mappedDemand)

In [ ]:
plotterNew = ValuesPlotter(valuesGrid)
plotterNew.plot_service_levels(ServiceType.School, gridDensity=200) # plots with griddata+contourf

In [46]:
class KPICalculator:
    '''Class to aggregate demand and evaluate section based and position based KPIs'''
    
    def __init__(self, demandUnits, serviceUnits):
        assert all([isinstance(du, DemandUnit) for du in demandUnits]),'Demand unit list expected'
        assert all([isinstance(su, ServiceUnit) for su in serviceUnits]),'Demand unit list expected'
        self.demand = demandUnits
        self.sources = serviceUnits
        
        # initialise the service evaluator
        self.evaluator = ServiceEvaluator(serviceUnits)
        self.serviceValues = {}
        self.quartiereKPI = {}
        
        # extract the mapped positions from demand units
        self.mappedPositions = MappedPositionsFrame(
            positions = [u.position for u in self.demand],
            idQuartiere= [u.attributes.get(common_cfg.IdQuartiereColName, [np.nan]) \
                          for u in self.demand])
        
        # stack ages distributions in a frame
        self.agesFrame = pd.concat([u.export for u in self.demand])
        
        # store frequently used properties
        #self.services = list(self.evaluator.outputServices)
        #self.quartieri = self.mappedPositions.index.levels[0]
        
    def evaluate_services_at_demand(self):
        self.serviceValues = self.evaluator.evaluate_services_at(self.mappedPositions)
        return self.serviceValues
    
    
    def compute_kpi_for_localized_services(self):
        assert self.serviceValues, 'Service values not available, have you computed them?'
        # get mean service levels by quartiere, weighting according to the number of citizens
        for service, data in self.serviceValues.items():
            out = []
            for col in tt.agesFrame.columns: # iterate over columns as Enums are not orderable...
                out.append(pd.Series.multiply(
                    data[col], tt.agesFrame[col], level=1)/tt.agesFrame[col].sum())
            weightedData = pd.concat(out)
            # assign to output
            self.quartiereKPI[service] = weightedData.groupby(
                common_cfg.IdQuartiereColName).mean()
        
        return self.quartiereKPI
    
    
    def compute_kpi_for_istat_values(self):
        pass
    
        

In [ ]:
demandTest = unitsOut[:10]
tt = KPICalculator(demandTest, schoolUnits[:10])
tt.evaluate_services_at_demand()
tt.compute_kpi_for_localized_services()

In [ ]:
tt.quartiereKPI